# Graph PaySim
This notebook is an example of using Neo4j with Vertex AI.  It takes PaySim data from a Neo4j database, puts that into Feature Store and then runs two classifications on that.  One classification uses the standard PaySim features.  A second uses features engineered using Neo4j Graph Data Science.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/benofben/vertex-ai-samples/blob/master/notebooks/community/neo4j/graph_paysim.ipynb" target="_parent">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/benofben/vertex-ai-samples/tree/master/notebooks/community/neo4j/graph_paysim.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Data Set
The notebook uses a version of the PaySim dataset that has been modified to work with Neo4j's graph database.  PaySim is a synthetic fraud dataset.  The goal is to identify whether or not a given transaction constitutes fraud.  The dataset is [here](https://github.com/voutilad/PaySim).

## Prerequisites
We assume that you've already loaded the PaySim data into a Neo4j instance and have the credentials to connect to that.

In [1]:
!pip install neo4j

     |████████████████████████████████| 76 kB 1.9 MB/s 
  Created wheel for neo4j: filename=neo4j-4.3.7-py3-none-any.whl size=100642 sha256=8e601bae45150fec8b59047a1a0432775a075717ae36ef5391a137e46477db78
  Stored in directory: /root/.cache/pip/wheels/b5/24/bb/cece9fcfdd5e1aa0683e2533945e1e3f27f70f342ff7e28993
Successfully built neo4j


In [ ]:
!pip install google.cloud.aiplatform

## Working with Neo4j
In this section we're going to connect to Neo4j and look around the database.  We're going to generate some new features in the dataset using Neo4j's Graph Data Science library.  Finally, we'll load the data into a Pandas dataframe so that it's all ready to put into GCP Feature Store.

In [2]:
import pandas as pd
from neo4j import GraphDatabase

In [3]:
DB_ULR = 'neo4j+s://6c443062.databases.neo4j.io:7687'
DB_USER = 'neo4j'
DB_PASS = 'some password'
DB_NAME = 'neo4j'

In [4]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))

Now, let's explore the data in the database a bit to understand what we have to work with.

In [5]:
# node labels
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL db.labels() YIELD label
    CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as freq', {})
    YIELD value
    RETURN label, value.freq AS freq
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,label,freq
0,Node,0
1,Client,11270
2,Bank,5
3,Merchant,3465
4,Mule,0
5,CashIn,746751
6,CashOut,424574
7,Debit,130284
8,Payment,542443
9,Transfer,0


In [6]:
# relationship types
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
      """
      CALL db.relationshipTypes() YIELD relationshipType as type
      CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as freq', {})
      YIELD value
      RETURN type AS relationshipType, value.freq AS freq
      ORDER by freq DESC
      """
      ).data()
    )
df = pd.DataFrame(result)
display(df)

,relationshipType,freq
0,PERFORMED,1844052
1,TO,1844052
2,NEXT,1833720
3,HAS_SSN,11330
4,HAS_EMAIL,11330
5,HAS_PHONE,11330
6,FIRST_TX,10332
7,LAST_TX,10332


In [7]:
# transaction types
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    MATCH (t:Transaction)
    WITH sum(t.amount) AS globalSum, count(t) AS globalCnt
    WITH *, 10^3 AS scaleFactor
    UNWIND ['CashIn', 'CashOut', 'Payment', 'Debit', 'Transfer'] AS txType
      CALL apoc.cypher.run('MATCH (t:' + txType + ')
        RETURN sum(t.amount) as txAmount, count(t) AS txCnt', {})
      YIELD value
    RETURN txType,value.txAmount AS TotalMarketValue
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,txType,TotalMarketValue
0,CashIn,1.040582e+11
1,CashOut,5.385410e+10
2,Payment,9.646814e+10
3,Debit,1.016829e+09
4,Transfer,0.000000e+00


## Create a New Feature with a Graph Embedding using Neo4j
First we're going to create an in memory graph represtation of the data in Neo4j Graph Data Science (GDS).

In [9]:
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL gds.graph.create.cypher('client_graph', 
      'MATCH (c:Client) RETURN id(c) as id, c.num_transactions as num_transactions, c.total_transaction_amnt as total_dollar_amnt, c.is_fraudster as is_fraudster',
      'MATCH (c:Client)-[:PERFORMED]->(t:Transaction)-[:TO]->(c2:Client) return id(c) as source, id(c2) as target, sum(t.amount) as amount, "TRANSACTED_WITH" as type ')
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,createMillis
0,"MATCH (c:Client) RETURN id(c) as id, c.num_tra...",MATCH (c:Client)-[:PERFORMED]->(t:Transaction)...,client_graph,11270,26035,370


Now we can generate an embedding from that graph.  This is a new feature we can use in our predictions.  We're using FastRP, which is a more full featured and higher performance of Node2Vec.  You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

In [10]:
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL gds.fastRP.mutate('client_graph',{
      relationshipWeightProperty:'amount',
      iterationWeights: [0.0, 1.00, 1.00, 0.80, 0.60],
      featureProperties: ['num_transactions', 'total_dollar_amnt'],
      propertyRatio: .25, 
      embeddingDimension:16,
      randomSeed: 1, 
      mutateProperty:'embedding'
    })
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,nodePropertiesWritten,mutateMillis,nodeCount,createMillis,computeMillis,configuration
0,11270,0,11270,0,14,"{'relationshipWeightProperty': 'amount', 'prop..."


Finally we dump that out to a dataframe

In [11]:
with driver.session(database = DB_NAME) as session:
  result = session.read_transaction( lambda tx: 
    tx.run(
    """
    CALL gds.graph.streamNodeProperties
    ('client_graph', ['embedding', 'num_transactions', 'total_dollar_amnt', 'is_fraudster'])
    YIELD nodeId, nodeProperty, propertyValue
    RETURN nodeId, nodeProperty, propertyValue
    """
    ).data()
  )
  df = pd.DataFrame(result)
  display(df)

,nodeId,nodeProperty,propertyValue
0,0,embedding,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,num_transactions,4
2,0,total_dollar_amnt,118919
3,0,is_fraudster,1
4,3,embedding,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
45075,1858717,is_fraudster,10000
45076,1858725,embedding,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
45077,1858725,num_transactions,1
45078,1858725,total_dollar_amnt,1557.3


Now we need to take that dataframe and shape it into something that better represents our classification problem.

In [12]:
x = df.pivot(index='nodeId', columns='nodeProperty', values='propertyValue')
x = x.reset_index()
x.columns.name = None
x

,nodeId,embedding,is_fraudster,num_transactions,total_dollar_amnt
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,4,118919
1,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,0,0
2,5,"[6.206621439019955e-09, -1.9226471081879026e-0...",1,80,7.48446e+06
3,8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,0,0
4,10,"[7.890789133213616e-10, -0.021041372790932655,...",1,227,3.75806e+07
...,...,...,...,...,...
11265,1857442,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,1,21671
11266,1857502,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,1,93.5579
11267,1857640,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,1,92.3323
11268,1858717,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000,1,11272.2


is_fraudster will have a value of 0 or 1 if populated.  If the value is 10000 then it's unlabled, so we're going to drop it.

In [13]:
x = x.loc[x['is_fraudster'] != 10000]
x

,nodeId,embedding,is_fraudster,num_transactions,total_dollar_amnt
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,4,118919
2,5,"[6.206621439019955e-09, -1.9226471081879026e-0...",1,80,7.48446e+06
4,10,"[7.890789133213616e-10, -0.021041372790932655,...",1,227,3.75806e+07
6,15,"[3.494107261303725e-07, -0.005100565031170845,...",1,106,4.86428e+06
7,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,0
...,...,...,...,...,...
11192,1839904,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,1,52.3779
11204,1844183,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,1,133989
11224,1847685,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,1,27.6889
11242,1849983,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,1,194573


Note that the embedding row is an array.  To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.

In [14]:
y = pd.DataFrame()
for index, row in x.iterrows():
  embedding = row['embedding']
  i=0
  for column in embedding:
    row['embedding_' + str(i)] = column
    i+=1
  y = y.append(row)
y = y.drop(columns = 'embedding')
y

,embedding_0,embedding_1,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,is_fraudster,nodeId,num_transactions,total_dollar_amnt
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.0,4.0,1.189191e+05
2,6.206621e-09,-1.922647e-08,-2.954485e-09,3.013137e-09,-1.699997,-1.700010,1.699997,1.699997,-1.701400e-08,-1.456331e-08,-2.666907e-09,3.647712e-09,-4.169740e-09,3.883921e-09,1.726210e-08,-1.442420e-08,1.0,5.0,80.0,7.484460e+06
4,7.890789e-10,-2.104137e-02,6.377600e-08,2.167623e-09,-1.699563,-1.699574,1.699563,1.699563,6.245183e-09,1.153816e-09,-2.104137e-02,2.622337e-09,-2.104137e-02,2.104136e-02,2.104136e-02,-1.408821e-09,1.0,10.0,227.0,3.758064e+07
6,3.494107e-07,-5.100565e-03,5.130220e-03,-5.351739e-03,-1.699944,-1.699967,1.699944,1.699944,-5.127179e-03,-4.834618e-03,-4.943346e-03,1.036349e-02,5.304334e-03,5.348149e-05,-5.125498e-03,3.505985e-05,1.0,15.0,106.0,4.864282e+06
7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,18.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11192,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1839904.0,1.0,5.237794e+01
11204,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1844183.0,1.0,1.339891e+05
11224,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1847685.0,1.0,2.768894e+01
11242,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1849983.0,1.0,1.945731e+05


And that's it!  The dataframe now has a nice dataset that we can use with GCP Vertex AI.

##Authenticate to GCP
In these steps, we'll connect to GCP.  Note that you may have to do this multiple times if you restart your environment.

In [4]:
from google.colab import auth
auth.authenticate_user()

In [6]:
import os
os.environ['GCLOUD_PROJECT'] = 'Neo4jBusinessDev'

##Upload to a GCP Cloud Storage Bucket

To get the data into Vertex AI, we must first put it in a bucket as a CSV.

In [15]:
filename = 'clients.csv'
y.to_csv(filename, index=False)

In [18]:
from google.cloud import storage
client = storage.Client()

In [21]:
# Create a bucket
bucket_name='paysim'
bucket = client.bucket(bucket_name)
new_bucket = client.create_bucket(bucket)

In [22]:
# Upload our CSV to that bucket
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(filename)
blob.upload_from_filename(filename)

## Create a Dataset with Vertex AI
Now we're going to take the CSV in our bucket and turn that into a dataset in Vertex AI

In [7]:
from google.cloud import aiplatform
aiplatform.init()

dataset = aiplatform.TabularDataset.create(display_name='paysim', gcs_source='gs://paysim/clients.csv')
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')


PermissionDenied: ignored

## Classification with Vertex AI
In this section, we're going to run two classifiers and compare results.  The first will use the standard PaySim features.  The second will use our new graph features.

## Analyze the Predictions
Now that Vertex AI has made predictions on the dataset, we're going to use Neo4j Bloom to investigate how those predictions fit with the data they were made from.

In [ ]:
pass

## Loading Data into GCP Feature Store
In this section, we'll take our dataframe with newly engineered features and load that into GCP feature store.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud.aiplatform_v1 import FeaturestoreServiceClient
API_ENDPOINT = 'us-central1-aiplatform.googleapis.com'
admin_client = FeaturestoreServiceClient(client_options={'api_endpoint': API_ENDPOINT})

PROJECT_ID = 'Neo4jBusinessDev'
REGION = 'us-central1'
BASE_RESOURCE_PATH = admin_client.common_location_path(PROJECT_ID, REGION)

In [ ]:
from google.cloud.aiplatform_v1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1.types import featurestore as featurestore_pb2
    
FEATURESTORE_ID = "paysim"
create_lro = admin_client.create_featurestore(
    featurestore_service_pb2.CreateFeaturestoreRequest(
        parent=BASE_RESOURCE_PATH,
        featurestore_id=FEATURESTORE_ID,
        featurestore=featurestore_pb2.Featurestore(
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=1
            ),
        ),
    )
)

PermissionDenied: ignored

In [ ]:

worker_count = 2
location = 'us-central1'
timeout = 300,

#client_options = {'api_endpoint': 'us-central1-aiplatform.googleapis.com'}
client = aiplatform.gapic.FeaturestoreServiceClient()

'''
entity_type = f"projects/{project}/locations/{location}/featurestores/{featurestore_id}/entityTypes/{entity_type_id}"
avro_source = aiplatform.gapic.AvroSource(gcs_source=aiplatform.gapic.GcsSource(uris=[avro_gcs_uri]))
    
feature_specs = [
  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id='embedding'),
  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id='is_fraudster'),
  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id='num_transactions'),
  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id='total_dollar_amnt'),
]

import_feature_values_request = aiplatform.gapic.ImportFeatureValuesRequest(
  entity_type=entity_type,
  avro_source=avro_source,
  feature_specs=feature_specs,
  entity_id_field=entity_id_field,
  feature_time_field=feature_time_field,
  worker_count=worker_count,
)
lro_response = client.import_feature_values(request=import_feature_values_request)
print("Long running operation:", lro_response.operation.name)
import_feature_values_response = lro_response.result(timeout=timeout)
print("import_feature_values_response:", import_feature_values_response)
'''

'\nentity_type = f"projects/{project}/locations/{location}/featurestores/{featurestore_id}/entityTypes/{entity_type_id}"\navro_source = aiplatform.gapic.AvroSource(gcs_source=aiplatform.gapic.GcsSource(uris=[avro_gcs_uri]))\n    \nfeature_specs = [\n  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id=\'embedding\'),\n  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id=\'is_fraudster\'),\n  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id=\'num_transactions\'),\n  aiplatform.gapic.ImportFeatureValuesRequest.FeatureSpec(id=\'total_dollar_amnt\'),\n]\n\nimport_feature_values_request = aiplatform.gapic.ImportFeatureValuesRequest(\n  entity_type=entity_type,\n  avro_source=avro_source,\n  feature_specs=feature_specs,\n  entity_id_field=entity_id_field,\n  feature_time_field=feature_time_field,\n  worker_count=worker_count,\n)\nlro_response = client.import_feature_values(request=import_feature_values_request)\nprint("Long running operation:", lro_response.operatio

##Cleanup

To delete the Graph Data Science representation of the graph, run this:

In [8]:
with driver.session(database = DB_NAME) as session:
    result = session.read_transaction( lambda tx: 
        tx.run(
        """
        CALL gds.graph.drop('client_graph')
        """
        ).data()
    )